# Neural Machine Translation with Attention : Dates

Building a Neural network model to translate human-readable dates like ("30th of June, 2019") into machine-readable dates like (30-06-2019).

1 . Importing All Necessary Libraries

In [1]:
from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K
import numpy as np
from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Using TensorFlow backend.


Reading the generated file of dates which we generated from another file

In [2]:
data= np.load(r'C:\Users\Eswar\Desktop\Simpli_Learn\Internship\Date_data.npy')

In [3]:
human_vocab = {' ': 0, '.': 1, '/': 2, '0': 3, '1': 4, '2': 5, '3': 6, '4': 7, '5': 8, '6': 9, '7': 10, '8': 11,
                   '9': 12, 'a': 13, 'b': 14, 'c': 15, 'd': 16, 'e': 17, 'f': 18, 'g': 19, 'h': 20, 'i': 21, 'j': 22,
                   'l': 23, 'm': 24, 'n': 25, 'o': 26, 'p': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'w': 33,
                   'y': 34, '<unk>': 35, '<pad>': 36}
machine_vocab = {'-': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10}
inv_machine_vocab = {0: '-', 1: '0', 2: '1', 3: '2', 4: '3', 5: '4', 6: '5', 7: '6', 8: '7', 9: '8', 10: '9'}

Preprocessing Data

In [4]:
def processing_data(data, human_vocab, machine_vocab, Tx, Ty):
    X, Y = zip(*data)
    
    X = np.array([string_to_int(i, Tx, human_vocab) for i in X])
    Y = [string_to_int(t, Ty, machine_vocab) for t in Y]
    
    Xoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), X)))
    Yoh = np.array(list(map(lambda x: to_categorical(x, num_classes=len(machine_vocab)), Y)))

    return X, np.array(Y), Xoh, Yoh

In [5]:
def string_to_int(string, length, vocab):
    string = string.lower()
    string = string.replace(',','')
    
    if len(string) > length:
        string = string[:length]
        
    rep = list(map(lambda x: vocab.get(x, '<unk>'), string))
    
    if len(string) < length:
        rep += [vocab['<pad>']] * (length - len(string))
    
    return rep

In [6]:
string_to_int('April 8, 2000', 15, human_vocab)

[13, 27, 28, 21, 23, 0, 11, 0, 5, 3, 3, 3, 36, 36, 36]

In [7]:
Tx = 15
Ty = 10
X, Y, Xoh, Yoh = processing_data(data, human_vocab, machine_vocab, Tx, Ty)

print("X.shape:", X.shape)
print("Y.shape:", Y.shape)
print("Xoh.shape:", Xoh.shape)
print("Yoh.shape:", Yoh.shape)

X.shape: (30000, 15)
Y.shape: (30000, 10)
Xoh.shape: (30000, 15, 37)
Yoh.shape: (30000, 10, 11)


Neural Network Building

In [8]:
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation('softmax', name='attention_weights')
dotor = Dot(axes = 1)

In [9]:
def attention(a, s_prev):
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    
    return context

In [10]:
n_a = 32
n_s = 64
post_activation_LSTM_cell = LSTM(n_s, return_state = True)
output_layer = Dense(len(machine_vocab), activation='softmax')

In [11]:
def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):
    X = Input(shape=(Tx, human_vocab_size))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    outputs = []
    
    a = Bidirectional(LSTM(n_a, return_sequences = True))(X)
    
    for t in range(Ty):
        context = attention(a, s)
        s, _, c = post_activation_LSTM_cell(context, initial_state=[s, c])
        out = output_layer(s)
        outputs.append(out)
    
    model = Model([X, s0, c0], outputs)
    return model

In [12]:
mod = model(Tx, Ty, n_a, n_s, len(human_vocab), len(machine_vocab))

In [13]:
opt = Adam(lr=0.005, beta_1=0.9, beta_2=0.999, decay=0.01)
mod.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
s0 = np.zeros((30000, n_s))
c0 = np.zeros((30000, n_s))
outputs = list(Yoh.swapaxes(0,1))
mod.fit([Xoh, s0, c0], outputs, epochs=5, batch_size=100)


Epoch 1/5


 2400/30000 [=>............................] - ETA: 1:27:24 - loss: 24.5956 - dense_3_loss: 2.4315 - dense_3_accuracy: 0.0000e+00 - dense_3_accuracy_1: 0.0800 - dense_3_accuracy_2: 0.2400 - dense_3_accuracy_3: 0.1100 - dense_3_accuracy_4: 0.0000e+00 - dense_3_accuracy_5: 0.0600 - dense_3_accuracy_6: 0.0500 - dense_3_accuracy_7: 0.0000e+00 - dense_3_accuracy_8: 0.0700 - dense_3_accuracy_9: 0.08 - ETA: 43:54 - loss: 23.7284 - dense_3_loss: 2.4313 - dense_3_accuracy: 0.0350 - dense_3_accuracy_1: 0.0600 - dense_3_accuracy_2: 0.1400 - dense_3_accuracy_3: 0.0650 - dense_3_accuracy_4: 0.4050 - dense_3_accuracy_5: 0.0900 - dense_3_accuracy_6: 0.0500 - dense_3_accuracy_7: 0.3900 - dense_3_accuracy_8: 0.0700 - dense_3_accuracy_9: 0.0600             - ETA: 29:24 - loss: 23.1553 - dense_3_loss: 2.5068 - dense_3_accuracy: 0.0233 - dense_3_accuracy_1: 0.0467 - dense_3_accuracy_2: 0.1033 - dense_3_accuracy_3: 0.0433 - dense_3_accuracy_4: 0.5933 - dense_3_accuracy_5: 0.0700 - dense_3_accuracy_6: 0.033

 7400/30000 [======>.......................] - ETA: 2:10 - loss: 17.7473 - dense_3_loss: 2.5272 - dense_3_accuracy: 0.4548 - dense_3_accuracy_1: 0.7484 - dense_3_accuracy_2: 0.3864 - dense_3_accuracy_3: 0.1244 - dense_3_accuracy_4: 0.7032 - dense_3_accuracy_5: 0.1688 - dense_3_accuracy_6: 0.0996 - dense_3_accuracy_7: 0.5660 - dense_3_accuracy_8: 0.2586 - dense_3_accuracy_9: 0.132 - ETA: 2:08 - loss: 17.6949 - dense_3_loss: 2.5195 - dense_3_accuracy: 0.4622 - dense_3_accuracy_1: 0.7496 - dense_3_accuracy_2: 0.3890 - dense_3_accuracy_3: 0.1233 - dense_3_accuracy_4: 0.7078 - dense_3_accuracy_5: 0.1678 - dense_3_accuracy_6: 0.0998 - dense_3_accuracy_7: 0.5647 - dense_3_accuracy_8: 0.2600 - dense_3_accuracy_9: 0.135 - ETA: 2:05 - loss: 17.6321 - dense_3_loss: 2.5125 - dense_3_accuracy: 0.4710 - dense_3_accuracy_1: 0.7533 - dense_3_accuracy_2: 0.3906 - dense_3_accuracy_3: 0.1244 - dense_3_accuracy_4: 0.7115 - dense_3_accuracy_5: 0.1673 - dense_3_accuracy_6: 0.1013 - dense_3_accuracy_7: 0.561

 9900/30000 [========>.....................] - ETA: 1:34 - loss: 16.4870 - dense_3_loss: 2.3681 - dense_3_accuracy: 0.5985 - dense_3_accuracy_1: 0.7963 - dense_3_accuracy_2: 0.4363 - dense_3_accuracy_3: 0.1325 - dense_3_accuracy_4: 0.7900 - dense_3_accuracy_5: 0.1808 - dense_3_accuracy_6: 0.1119 - dense_3_accuracy_7: 0.5153 - dense_3_accuracy_8: 0.3003 - dense_3_accuracy_9: 0.180 - ETA: 1:33 - loss: 16.4424 - dense_3_loss: 2.3613 - dense_3_accuracy: 0.6029 - dense_3_accuracy_1: 0.7982 - dense_3_accuracy_2: 0.4380 - dense_3_accuracy_3: 0.1330 - dense_3_accuracy_4: 0.7924 - dense_3_accuracy_5: 0.1843 - dense_3_accuracy_6: 0.1121 - dense_3_accuracy_7: 0.5167 - dense_3_accuracy_8: 0.3004 - dense_3_accuracy_9: 0.182 - ETA: 1:31 - loss: 16.3963 - dense_3_loss: 2.3551 - dense_3_accuracy: 0.6071 - dense_3_accuracy_1: 0.7999 - dense_3_accuracy_2: 0.4394 - dense_3_accuracy_3: 0.1336 - dense_3_accuracy_4: 0.7948 - dense_3_accuracy_5: 0.1908 - dense_3_accuracy_6: 0.1125 - dense_3_accuracy_7: 0.516

12400/30000 [===========>..................] - ETA: 1:10 - loss: 15.1022 - dense_3_loss: 2.2151 - dense_3_accuracy: 0.6747 - dense_3_accuracy_1: 0.8240 - dense_3_accuracy_2: 0.4728 - dense_3_accuracy_3: 0.1456 - dense_3_accuracy_4: 0.8412 - dense_3_accuracy_5: 0.3488 - dense_3_accuracy_6: 0.1361 - dense_3_accuracy_7: 0.6033 - dense_3_accuracy_8: 0.3520 - dense_3_accuracy_9: 0.225 - ETA: 1:10 - loss: 15.0453 - dense_3_loss: 2.2092 - dense_3_accuracy: 0.6767 - dense_3_accuracy_1: 0.8247 - dense_3_accuracy_2: 0.4731 - dense_3_accuracy_3: 0.1462 - dense_3_accuracy_4: 0.8428 - dense_3_accuracy_5: 0.3545 - dense_3_accuracy_6: 0.1378 - dense_3_accuracy_7: 0.6071 - dense_3_accuracy_8: 0.3539 - dense_3_accuracy_9: 0.227 - ETA: 1:09 - loss: 14.9862 - dense_3_loss: 2.2033 - dense_3_accuracy: 0.6784 - dense_3_accuracy_1: 0.8250 - dense_3_accuracy_2: 0.4740 - dense_3_accuracy_3: 0.1473 - dense_3_accuracy_4: 0.8443 - dense_3_accuracy_5: 0.3598 - dense_3_accuracy_6: 0.1405 - dense_3_accuracy_7: 0.611

...

22400/30000 [=====================>........] - ETA: 18s - loss: 2.9588 - dense_3_loss: 0.5531 - dense_3_accuracy: 0.9137 - dense_3_accuracy_1: 0.9141 - dense_3_accuracy_2: 0.7582 - dense_3_accuracy_3: 0.6741 - dense_3_accuracy_4: 0.9999 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9551 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8555 - dense_3_accuracy_9: 0.806 - ETA: 17s - loss: 2.9596 - dense_3_loss: 0.5531 - dense_3_accuracy: 0.9135 - dense_3_accuracy_1: 0.9140 - dense_3_accuracy_2: 0.7582 - dense_3_accuracy_3: 0.6739 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9552 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8557 - dense_3_accuracy_9: 0.806 - ETA: 17s - loss: 2.9599 - dense_3_loss: 0.5527 - dense_3_accuracy: 0.9134 - dense_3_accuracy_1: 0.9140 - dense_3_accuracy_2: 0.7580 - dense_3_accuracy_3: 0.6738 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9551 - dense_3_accuracy_7: 0.9999 - de

24900/30000 [=======================>......] - ETA: 13s - loss: 2.9461 - dense_3_loss: 0.5497 - dense_3_accuracy: 0.9137 - dense_3_accuracy_1: 0.9146 - dense_3_accuracy_2: 0.7583 - dense_3_accuracy_3: 0.6736 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9880 - dense_3_accuracy_6: 0.9554 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8576 - dense_3_accuracy_9: 0.807 - ETA: 13s - loss: 2.9444 - dense_3_loss: 0.5497 - dense_3_accuracy: 0.9139 - dense_3_accuracy_1: 0.9147 - dense_3_accuracy_2: 0.7584 - dense_3_accuracy_3: 0.6739 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9556 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8576 - dense_3_accuracy_9: 0.807 - ETA: 13s - loss: 2.9431 - dense_3_loss: 0.5493 - dense_3_accuracy: 0.9140 - dense_3_accuracy_1: 0.9149 - dense_3_accuracy_2: 0.7584 - dense_3_accuracy_3: 0.6740 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9556 - dense_3_accuracy_7: 0.9999 - de

27400/30000 [==========================>...] - ETA: 9s - loss: 2.9453 - dense_3_loss: 0.5493 - dense_3_accuracy: 0.9143 - dense_3_accuracy_1: 0.9155 - dense_3_accuracy_2: 0.7581 - dense_3_accuracy_3: 0.6740 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9882 - dense_3_accuracy_6: 0.9547 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8573 - dense_3_accuracy_9: 0.80 - ETA: 8s - loss: 2.9420 - dense_3_loss: 0.5487 - dense_3_accuracy: 0.9146 - dense_3_accuracy_1: 0.9157 - dense_3_accuracy_2: 0.7582 - dense_3_accuracy_3: 0.6745 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9881 - dense_3_accuracy_6: 0.9548 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8576 - dense_3_accuracy_9: 0.80 - ETA: 8s - loss: 2.9409 - dense_3_loss: 0.5485 - dense_3_accuracy: 0.9145 - dense_3_accuracy_1: 0.9157 - dense_3_accuracy_2: 0.7583 - dense_3_accuracy_3: 0.6745 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9882 - dense_3_accuracy_6: 0.9550 - dense_3_accuracy_7: 0.9999 - dense_3

29900/30000 [============================>.] - ETA: 4s - loss: 2.9367 - dense_3_loss: 0.5473 - dense_3_accuracy: 0.9151 - dense_3_accuracy_1: 0.9165 - dense_3_accuracy_2: 0.7579 - dense_3_accuracy_3: 0.6737 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9882 - dense_3_accuracy_6: 0.9546 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8582 - dense_3_accuracy_9: 0.80 - ETA: 4s - loss: 2.9352 - dense_3_loss: 0.5470 - dense_3_accuracy: 0.9152 - dense_3_accuracy_1: 0.9166 - dense_3_accuracy_2: 0.7579 - dense_3_accuracy_3: 0.6738 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9882 - dense_3_accuracy_6: 0.9548 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8583 - dense_3_accuracy_9: 0.80 - ETA: 4s - loss: 2.9354 - dense_3_loss: 0.5472 - dense_3_accuracy: 0.9152 - dense_3_accuracy_1: 0.9165 - dense_3_accuracy_2: 0.7579 - dense_3_accuracy_3: 0.6735 - dense_3_accuracy_4: 1.0000 - dense_3_accuracy_5: 0.9882 - dense_3_accuracy_6: 0.9548 - dense_3_accuracy_7: 0.9999 - dense_3

30000/30000 [==============================] - 52s 2ms/step - loss: 2.9293 - dense_3_loss: 0.5471 - dense_3_accuracy: 0.9152 - dense_3_accuracy_1: 0.9165 - dense_3_accuracy_2: 0.7589 - dense_3_accuracy_3: 0.6738 - dense_3_accuracy_4: 0.9999 - dense_3_accuracy_5: 0.9883 - dense_3_accuracy_6: 0.9546 - dense_3_accuracy_7: 0.9999 - dense_3_accuracy_8: 0.8585 - dense_3_accuracy_9: 0.8078


Testing the output 

Lets check for the date 30th of June, 2019

In [19]:
human_date= ('30th of june,2019')
source = string_to_int(human_date, Tx, human_vocab)
source = np.array(list(map(lambda x: to_categorical(x, num_classes=len(human_vocab)), source)))
source = source.reshape((1, ) + source.shape)
prediction = mod.predict([source, s0, c0])
prediction = np.argmax(prediction, axis = -1)
output = [inv_machine_vocab[int(i)] for i in prediction]
print("output:", ''.join(output))

output: 2019-06-30


In this way translated human-readable dates like ("30th of June, 2019") into machine-readable dates like (30-06-2019)